In [1]:
from tensorflow.keras.applications import *
from tensorflow.keras import models
from tensorflow.keras import layers
import pathlib
import os
import shutil
import pandas as pd
from random import randrange
from tensorflow.keras import optimizers
from sklearn import model_selection
import tensorflow as tf
import numpy as np

In [2]:

train_dir = r'C:\Users\superuser\Documents\DL\train\merged'
train_dir = pathlib.Path(train_dir)
test_dir = r'C:\Users\superuser\Documents\DL\test\merged'
test_dir = pathlib.Path(test_dir)

batch_size = 32
img_height = 360
img_width = 640
input_shape=(img_height, img_width, 3)

seed = randrange(1000)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split = 0.6,
  label_mode = 'categorical',
  subset = 'validation',
  seed = seed,
  smart_resize = True,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  validation_split = 0.2,
  label_mode = 'categorical',
  subset = 'validation',
  seed = seed,
  smart_resize = True,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
n_classes = len(class_names)
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 27888 files belonging to 4 classes.
Using 16732 files for validation.
Found 9923 files belonging to 4 classes.
Using 1984 files for validation.
['DL1', 'DL2', 'DL3', 'DL_BG']


In [3]:
conv_base = EfficientNetB1(weights='imagenet',include_top=False,input_shape=input_shape)

In [4]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name='gap'))
model.add(layers.Dropout(rate=0.2,name='dropout'))
model.add(layers.Dense(n_classes,activation='softmax',name='fc_out'))
conv_base.trainable = False
model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb1 (Functional)  (None, 12, 20, 1280)      6575239   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
fc_out (Dense)               (None, 4)                 5124      
Total params: 6,580,363
Trainable params: 5,124
Non-trainable params: 6,575,239
_________________________________________________________________


In [5]:
epochs=10

history = model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=epochs
)

Epoch 1/10
523/523 [==============================] - 885s 1s/step - loss: 1.4662 - accuracy: 0.7283 - val_loss: 0.4135 - val_accuracy: 0.8997
Epoch 2/10
523/523 [==============================] - 521s 996ms/step - loss: 0.5671 - accuracy: 0.8760 - val_loss: 0.1214 - val_accuracy: 0.9677
Epoch 3/10
523/523 [==============================] - 497s 951ms/step - loss: 0.4545 - accuracy: 0.8985 - val_loss: 0.1112 - val_accuracy: 0.9723
Epoch 4/10
523/523 [==============================] - 497s 950ms/step - loss: 0.4289 - accuracy: 0.9052 - val_loss: 0.1425 - val_accuracy: 0.9743
Epoch 5/10
523/523 [==============================] - 497s 950ms/step - loss: 0.3790 - accuracy: 0.9151 - val_loss: 0.1069 - val_accuracy: 0.9718
Epoch 6/10
523/523 [==============================] - 497s 950ms/step - loss: 0.3766 - accuracy: 0.9216 - val_loss: 0.0953 - val_accuracy: 0.9814
Epoch 7/10
523/523 [==============================] - 497s 950ms/step - loss: 0.3723 - accuracy: 0.9222 - val_loss: 0.0743 - va

In [6]:
model.save(r'C:\Users\superuser\Documents\DL\models\rotordet_net_v4')
np.save('my_history.npy',history.history)

C:\Users\superuser\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: C:\Users\superuser\Documents\DL\models\rotordet_net_v4\assets


NameError: name 'np' is not defined